In [1]:
import pandas as pd
import os 

In [2]:
path_to_metrics_csv = r'saving_dir\dlv3_res50pr-P1L-100ep-9cl-150-ch_[0-12]\version_0\metrics.csv'
full_results_df = pd.read_csv(path_to_metrics_csv)
train_cols = [c for c in full_results_df.columns if 'train' in c]
val_cols = [c for c in full_results_df.columns if 'val' in c]
train_results_df = full_results_df[['epoch']+train_cols].dropna()
val_results_df = full_results_df[['epoch']+val_cols].dropna()
idx = val_results_df['val_iou_mean'].argmax()
val_results_df.iloc[idx]

epoch                                51.000000
val_iou_UNLABELED                     0.786347
val_iou_buildings_territory           0.758021
val_iou_mean                          0.520549
val_iou_natural_ground                0.209921
val_iou_natural_grow                  0.217481
val_iou_natural_wetland               0.493314
val_iou_natural_wood                  0.816014
val_iou_quasi_natural_grow            0.593104
val_iou_transport                     0.145871
val_iou_water                         0.664867
val_loss                              0.964465
val_precision_UNLABELED               0.881722
val_precision_buildings_territory     0.859973
val_precision_mean                    0.695653
val_precision_natural_ground          0.600394
val_precision_natural_grow            0.466108
val_precision_natural_wetland         0.645462
val_precision_natural_wood            0.874642
val_precision_quasi_natural_grow      0.666602
val_precision_transport               0.405475
val_precision

# проверка правильности работы метрик

In [12]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

import numpy as np

def decode_confusion_matrix_2x2(confusion_matrix):
    tp = confusion_matrix[1, 1]
    tn = confusion_matrix[0, 0]
    fp = confusion_matrix[0, 1]
    fn = confusion_matrix[1, 0]
    return {'tp':tp, 'tn': tn, 'fp': fp, 'fn': fn}

def compute_accuracy_from_confusion(multiclass_confusion_matrix):
    confusion_sum = multiclass_confusion_matrix.sum(axis=0)
    tp, tn, fp, fn = decode_confusion_matrix_2x2(confusion_sum)
    accuracy = 0
    if tp+tn+fp+fn != 0:
        accuracy = (tp+tn)/(tp+tn+fp+fn)
    return accuracy

def compute_metric_from_confusion(multiclass_confusion_matrix, metric_params_dict, idx2class_name_dict=None):
    #print(f'metric_params_dict={metric_params_dict}')
    mean_metric = 0
    # {class_name: iou_val}
    metric_dict = {}
    actual_classes_num = 0
    metric_name = metric_params_dict['name']
    for idx, class_confusion in enumerate(multiclass_confusion_matrix):
        #print(f'class_conf_shape={class_confusion}')
        confusion_vals_dict = decode_confusion_matrix_2x2(class_confusion)
        #print(f'confusion_vals_dict={confusion_vals_dict}')
        #print(f'conf_sum={class_confusion.sum()};tn={confusion_vals_dict["tn"]}')
        if class_confusion.sum() != confusion_vals_dict['tn']:
            actual_classes_num += 1

        #print(f'actual_classes_num={actual_classes_num}')
        class_metric = 0
        numerator = [confusion_vals_dict[v] for v in metric_params_dict['numerator']]
        denominator = [confusion_vals_dict[v] for v in metric_params_dict['denominator']]
        #print(f'numerator={numerator}')
        #print(f'denominator={denominator}')
        if np.sum(denominator) != 0:
            class_metric = np.sum(numerator)/np.sum(denominator)
        mean_metric += class_metric
        class_name = f'{metric_name}_{idx}'
        if idx2class_name_dict is not None:
            class_name = f'{metric_name}_{idx2class_name_dict[idx]}'
        metric_dict[class_name] = class_metric
    if actual_classes_num == 0:
        metric_dict[f'{metric_name}_mean'] = 0
    else:
        metric_dict[f'{metric_name}_mean'] = mean_metric/actual_classes_num
    return metric_dict


In [25]:
np.random.seed(0)
class_num = 6
pred_labels = np.random.randint(0, class_num, size=[5000])
true_labels = np.random.randint(0, class_num, size=[5000])
confusion = metrics.multilabel_confusion_matrix(true_labels.reshape(-1), pred_labels.reshape(-1))
precision_results = metrics.precision_score(true_labels, pred_labels, average=None)
precision_params_dict = {'name':'precision', 'numerator':['tp'], 'denominator': ['tp', 'fp']}
precision_from_confusion = compute_metric_from_confusion(confusion,precision_params_dict,)

recall_results = metrics.recall_score(true_labels, pred_labels, average=None)
recall_params_dict = {'name':'recall', 'numerator':['tp'], 'denominator': ['tp', 'fn']}
recall_from_confusion = compute_metric_from_confusion(confusion,recall_params_dict,)

iou_results = metrics.jaccard_score(true_labels, pred_labels, average=None)
iou_params_dict = {'name':'iou', 'numerator':['tp'], 'denominator': ['tp', 'fp', 'fn']}
iou_from_confusion = compute_metric_from_confusion(confusion,iou_params_dict,)
recall_from_confusion, recall_results

({'recall_0': np.float64(0.15393939393939393),
  'recall_1': np.float64(0.15977653631284916),
  'recall_2': np.float64(0.15063291139240506),
  'recall_3': np.float64(0.16090584028605484),
  'recall_4': np.float64(0.1592814371257485),
  'recall_5': np.float64(0.16789215686274508),
  'recall_mean': np.float64(0.15873804598653277)},
 array([0.15393939, 0.15977654, 0.15063291, 0.16090584, 0.15928144,
        0.16789216]))